### 의사록 파일(.hwp)을 크롤링하여 새 디렉토리에 저장합니다.

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import re
from tika import parser
from os import listdir
from os.path import isfile, join

# 저장 위치 설정
SAVE_DB_DIR = "../data/hwp"
if not os.path.exists(SAVE_DB_DIR):
    os.makedirs(SAVE_DB_DIR)

# 전체 페이지 수 반환
def get_total_pages(url):
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content)
    end_page = int(soup.select('.i.end > a')[-1].attrs['href'].split('=')[-1])
    return end_page

# 페이지 내 데이터 크롤링
def crawl_page(url, page):
    params = {
        'pageIndex': page
    }
    resp = requests.get(url, params=params)
    soup = BeautifulSoup(resp.content)
    li_list = soup.select('.bdLine.type2 > ul > li')
    return li_list

# hwp 파일 다운로드
def download_hwp_files(li_list, save_dir):
    for li_item in li_list:
        link_li = li_item.select('.fileGoupBox ul li')
        for link in link_li:
            if link.select_one('a').attrs['title'][-3:] == 'hwp':
                link_url = link.select_one('a').attrs['href']
                title = li_item.select_one('.row span a span span').text
                download_url = 'http://www.bok.or.kr' + link_url
                file_res = requests.get(download_url)
                file_name = '{}.hwp'.format(title)
                file_path = os.path.join(save_dir, file_name)
                with open(file_path, 'wb') as f:
                    f.write(file_res.content)

# 메인 함수
def main():
    url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
    end_page = get_total_pages(url)
    print('총 {}페이지 까지 크롤링을 시작합니다.'.format(end_page))
    
    li_list = []
    for i in range(1, end_page + 1):
        li_list.extend(crawl_page(url, i))
        print(f"페이지 {i} 크롤링 성공")

    download_hwp_files(li_list, SAVE_DB_DIR)

if __name__ == "__main__":
    main()

### 저장한 파일의 텍스트를 추출한 후 데이터 정의 기준에 따라 전처리 과정을 진행한 후, .csv 파일 형태로 저장합니다.

In [ ]:
import re
from tika import parser
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
from datetime import datetime

# hwp 파일 텍스트로 변환
def convert_hwp_to_text(source_folder, output_folder):
    # 지정 폴더 내 파일 목록 조회 (파일만)
    hwp_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    
    result = []
    for hwp in hwp_files:
        time = re.search(r'\((.*?)\)', hwp).group(1)
        hwp_filepath = os.path.join(source_folder, hwp)
        
        # hwp 파일 텍스트로 변환
        parsedText = parser.from_file(hwp_filepath)["content"]
        new_dict = {
            'time': time,
            'text': parsedText
        }
        result.append(new_dict)

    df = pd.DataFrame(result)
    output_path = os.path.join(output_folder, '금통위의사록(텍스트파일).xlsx')

    # DataFrame을 Excel 파일로 저장
    df.to_excel(output_path, index=False)

# 텍스트 전처리 함수
def text_filtering(text):
    text = re.sub(r'(\n{2,}|- \d+ -|―|｢|｣|[․/→←+]|^.*hwp*\w*[A-Za-z])', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.split('6. 회의경과')[1]
    return text

# 메인 함수
def main():
    source_folder = "../data/hwp/"
    output_folder = "../data/"
    
    # hwp 파일을 텍스트로 변환하여 Excel로 저장
    convert_hwp_to_text(source_folder, output_folder)

    # Excel 파일 읽어오기
    minutes = pd.read_excel(os.path.join(output_folder, '금통위의사록(텍스트파일).xlsx'), usecols=[1, 2])
    minutes['time'] = minutes['time'].astype(str)

    # 날짜 추출 및 datetime으로 변환
    date_list = []
    for time in minutes['time']:
        date_list.extend(re.findall(r'\d{4}.\d{1,2}.\d{1,2}', time))
    minutes['time'] = date_list
    minutes['time'] = pd.to_datetime(minutes['time'], format='%Y.%m.%d')

    # 연도가 2009~2021년인 의사록 필터링
    minutes = minutes[(minutes['time'].dt.year >= 2009) & (minutes['time'].dt.year <= 2021)]
    minutes.sort_values(by='time')

    # 텍스트 전처리 적용
    minutes['text'] = minutes['text'].apply(text_filtering)

    # 결과를 csv 파일로 저장
    minutes.to_csv('의사록_hwp2text_전처리(완).csv', encoding='utf-8-sig', index=False)

if __name__ == "__main__":
    main()